<b>Audio and Music Processing Lab - Module 2</b><br>Rafael Caro Repetto<br>rafael.caro@upf.edu<br>15.02.2023
## AMPLab2 - Introduction to music21 (3)
### Exercise 1
Q. **How is instrumental accompaniment related to the vocal melody in jingju?**

M. *Plot a bar chart with the number of intervals formed between each note in the accompaniment and the corresponding one in the vocal melody*

### Test the proposed method with a single score

In [ ]:
from music21 import *
import matplotlib.pyplot as plt
import os

datasetPath = './Jingju Scores Dataset/MusicXML'# Path to the folder that contains the MusicXML scores

Since the idea is to count the intervals formed by the notes in the instrumental part with the notes at the vocal part, let's first extract all the notes from each part

In [ ]:
s = converter.parse(os.path.join(datasetPath, 'lseh-YiLunMing-WenZhaoGuan-1.xml'))

pi = s.parts[1] # Instrumental part
pv = s.parts[0] # Vocal part

ni = pi.flat.notes.stream() # All notes from the instrumental part
nv = pv.flat.notes.stream() # All notes for the vocal part

Then, let's count all the intervals in a dictionary. The intervals are formed by each note of the instrumental accompaniment and the one that is *sounding* at the same time (that is, at the same offset) in the vocal part, which might start at the same offset of the instrumental note, or can be sounding from before.

In [ ]:
intervals = {}

for n1 in ni:
    if not n1.duration.isGrace: # Skip grace notes
        o = n1.offset
        # Retrieve the notes in the vocal part that occur at the instrumental note's offset.
        # The mustBeginInSpan=False parameter allows retrieving notes that started before that
        # offset are still sounding at that position.
        # The result is a stream, which might contain more than one note in case there are
        # grace notes, which share the offset with the main note.
        nStr = nv.getElementsByOffset(o, mustBeginInSpan=False).stream()
        for n2 in nStr:
            if not n2.duration.isGrace: # Skip grace notes
                itv = interval.Interval(n1, n2)
                intervals[itv.name] = intervals.get(itv.name, 0) + 1

In [ ]:
intervals

In order to display a meaningful bar chart, order the intervals according to its size in semitones.

In [ ]:
# Create a dictionary with the equivalence of each interval's size in semitones and its name.
intervalsOrder = {}
for k in intervals.keys():
    itv = interval.Interval(k)
    intervalsOrder[itv.semitones] = k
    
# Ordered list of intervals by semitones size
xValues = sorted(intervalsOrder.keys())
# Oredred list of interval names by their semitiones size to be use as ticks for the x axis.
xTicks = [intervalsOrder[i] for i in xValues]
# Ordered list of y axis values
yValues = [intervals[i] for i in xTicks]

Plot the bar chart.

In [ ]:
plt.bar(xValues, yValues)
plt.xticks(xValues, xTicks)
plt.show()

### Apply the method to the whole dataset

Count intervals from the whole dataset

In [ ]:
intervals = {}

allScores = os.listdir(datasetPath)

for score in allScores:
    print('Parsing', score)
    s = converter.parse(os.path.join(datasetPath, score))
    
    ### Find out which parts are vocal and which instrumental
    
    vocalParts = [] # some scores have two vocal parts
    
    for p in s.parts:
        nn = p.flat.notes.stream() # to look for lyrics, only notes are needed
        if len(nn) > 0: # there are few scores with an empty part
            i = 0
            n = nn[i]
            # sometimes the vocal part starts with grace notes,
            # but lyrics are only attached to main notes
            while n.quarterLength == 0:
                i += 1
                n = nn[i]
            if n.lyric:
                vocalParts.append(p)
            else:
                pi = p

    ni = pi.flat.notes.stream() # All notes from the instrumental part
    
    for pv in vocalParts:
        nv = pv.flat.notes.stream() # All notes for the vocal part
        for n1 in ni:
            if n1.quarterLength > 0: # Skip grace notes
                o = n1.offset
                nStr = nv.getElementsByOffset(o, mustBeginInSpan=False).stream()
                for n2 in nStr:
                    if n2.quarterLength > 0: # Skip grace notes
                        itv = interval.Interval(n1, n2)
                        intervals[itv.name] = intervals.get(itv.name, 0) + 1

print('\nDone!')

Order the intervals by semitones and plot the histogram

In [ ]:
intervalsOrder = {}
for k in intervals.keys():
    itv = interval.Interval(k)
    intervalsOrder[itv.semitones] = k
    
xValues = sorted(intervalsOrder.keys())
xTicks = [intervalsOrder[i] for i in xValues]
yValues = [intervals[i] for i in xTicks]

plt.figure(figsize=(16,6))
plt.bar(xValues, yValues)
plt.xticks(xValues, xTicks)
plt.show()